In [2]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np
import matplotlib.pyplot as plt #Package for visualization
import re #importing package for Regular expression operations
from sklearn.model_selection import train_test_split #Package for splitting the data
from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical
from keras.preprocessing.text import Tokenizer #Tokenization
from keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils import to_categorical

In [3]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv('Sentiment.csv')

# Select only the necessary columns 'text' and 'sentiment'
mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]

In [5]:
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

<ipython-input-5-cee1da567eb8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(lambda x: x.lower())
<ipython-input-5-cee1da567eb8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))


In [6]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

In [7]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix


In [9]:
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons

In [10]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())

In [11]:
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split

In [12]:
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 41s - loss: 0.8267 - accuracy: 0.6418 - 41s/epoch - 141ms/step
144/144 - 2s - loss: 0.7571 - accuracy: 0.6721 - 2s/epoch - 11ms/step
0.757066547870636
0.6721275448799133


In [13]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


#Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [14]:
model.save('sentimentAnalysis.h5') #Saving the model

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [16]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [17]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")


1/1 - 0s - 266ms/epoch - 266ms/step
[0.7617619  0.08937758 0.14886051]
Neutral


#Apply GridSearchCV on the source code provided in the class

---



In [18]:
pip install scikeras

In [19]:
from scikeras.wrappers import KerasClassifier #importing Keras classifier

from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(model=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters

744/744 - 91s - loss: 0.8204 - accuracy: 0.6494 - 91s/epoch - 122ms/step
186/186 - 2s - 2s/epoch - 10ms/step


744/744 - 73s - loss: 0.8201 - accuracy: 0.6493 - 73s/epoch - 98ms/step
186/186 - 2s - 2s/epoch - 12ms/step


744/744 - 75s - loss: 0.8306 - accuracy: 0.6426 - 75s/epoch - 101ms/step
186/186 - 2s - 2s/epoch - 11ms/step


744/744 - 76s - loss: 0.8226 - accuracy: 0.6476 - 76s/epoch - 102ms/step
186/186 - 2s - 2s/epoch - 11ms/step


744/744 - 82s - loss: 0.8127 - accuracy: 0.6504 - 82s/epoch - 110ms/step
186/186 - 2s - 2s/epoch - 11ms/step


...

Epoch 1/2
186/186 - 27s - loss: 0.8400 - accuracy: 0.6392 - 27s/epoch - 147ms/step
Epoch 2/2
186/186 - 18s - loss: 0.6945 - accuracy: 0.7002 - 18s/epoch - 97ms/step
47/47 - 1s - 675ms/epoch - 14ms/step


Epoch 1/2
186/186 - 26s - loss: 0.8540 - accuracy: 0.6293 - 26s/epoch - 140ms/step
Epoch 2/2
186/186 - 19s - loss: 0.6943 - accuracy: 0.7026 - 19s/epoch - 100ms/step
47/47 - 1s - 667ms/epoch - 14ms/step


Epoch 1/2
186/186 - 25s - loss: 0.8455 - accuracy: 0.6364 - 25s/epoch - 135ms/step
Epoch 2/2
186/186 - 18s - loss: 0.6836 - accuracy: 0.7126 - 18s/epoch - 96ms/step
47/47 - 1s - 604ms/epoch - 13ms/step


Epoch 1/2
186/186 - 26s - loss: 0.8384 - accuracy: 0.6351 - 26s/epoch - 139ms/step
Epoch 2/2
186/186 - 17s - loss: 0.6732 - accuracy: 0.7107 - 17s/epoch - 89ms/step
47/47 - 1s - 914ms/epoch - 19ms/step


Epoch 1/2
930/930 - 95s - loss: 0.8094 - accuracy: 0.6492 - 95s/epoch - 102ms/step
Epoch 2/2
930/930 - 86s - loss: 0.6717 - accuracy: 0.7165 - 86s/epoch - 93ms/step
Best: 0.680512 using {'batch_size': 10, 'epochs': 2}
